In [1]:
import pandas as pd
import sys
import os 

# sys.path.append("../")
# sys.path.append("../../")
# sys.path.append("../../../")



In [1]:
from dotenv import dotenv_values

config = dotenv_values(".env.dev")

config["KREAM_DETAILS_TEMP_DIR"] 

'router/dev/kream/data/detail/temp/'

In [1]:
from router.dev.kream import create_last_update_kream_detail_log,get_kream_product_card

data  =create_last_update_kream_detail_log("231103-012830-adidas")

231103-012830 adidas


In [1]:
from router.dev.kream import get_kream_buy_and_sell
x = get_kream_buy_and_sell('adidas','231103-012830',sample=20)

In [6]:
from model.db_model_kream import KreamProductIdBridgeSchmea

x = KreamProductIdBridgeSchmea(kream_id=123,product_id='asd')

x = list(x)

data = [del row.product_id for row in x]

SyntaxError: invalid syntax (3360895286.py, line 7)

In [4]:
from datetime import datetime

# value = datetime.now().replace(microsecond=0)
value = "2023-09-15T00:00:00"
parsed_datetime = datetime.strptime(value, "%Y-%m-%dT%H:%M:%S")
print(parsed_datetime)

2023-09-15 00:00:00


In [8]:
import json


def update_scrap_result(scrap_name, key, value):
    path = "router/dev/kream/data/scrap-result/"
    assert path, "KREAM_SCRAP_RESULT_DIR is not defined in .env"

    with open(path + scrap_name + ".json", "r") as f:
        scrap_result = json.load(f)
        scrap_result[key] = value

    with open(path + scrap_name + ".json", "w") as f:
        f.write(json.dumps(scrap_result, default=str))

    return scrap_result


update_scrap_result("231106-011606-adidas","scrap_name","231106-011606-adidas")

{'scrap_name': '231106-011606-adidas',
 'brand': 'adidas',
 'scrap_time': '231106-011606',
 'num_process': 1,
 'ref_product_card': '177304',
 'scrap_result': {'177304': 'success'},
 'db_update': True,
 'kream_product_card': {'len': 1,
  'kream_id_list': [177304],
  'data': [{'kream_id': 177304,
    'kream_product_img_url': 'https://kream-phinf.pstatic.net/MjAyMzA5MThfNDMg/MDAxNjk1MDAxMTkyNzk0.MPhdvYhgKKaCiOfb8-wyaVblv7JhQeqafM4E3gA-wGMg.0FlQFPi80zCdb0q-G5U1cFQKMNfTT8NGqeR5eFr5c3Ig.PNG/a_0235bb34e55445cba5577cf6b3575a73.png?type=l',
    'kream_product_name': 'adidas gazelle core black gold metallic gum',
    'brand_name': 'adidas',
    'retail_price': 129000,
    'product_release_date': '2023-09-15 00:00:00',
    'color': 'core black/gold metallic/gum',
    'wish': 7321,
    'review': 7,
    'updated_at': '2023-11-06 01:15:49'}]},
 'kream_trading_volume': {'len': 1,
  'kream_id_list': [177304],
  'data': [{'kream_id': 177304,
    'kream_product_size': '260',
    'price': 133000,
    'li